In [1]:
# import packages 
import pandas as pd
import os
from io import StringIO
from skimage import io, transform
import numpy as np
import open3d as o3d
import copy
import napari
from skimage import morphology
from skimage.measure import label, regionprops, block_reduce
from scipy import stats, ndimage
import matplotlib.pyplot as plt
from tifffile import imsave
from tqdm import tqdm
import sys
import os


root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)


from src.aux_pcd_functions  import pcd_to_image, image_to_pcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def _manual_registration(source, source_values, target, target_values):
    
    mat_ground = o3d.visualization.Material("defaultLitSSR")
    mat_ground.scalar_properties['roughness'] = 0.15
    mat_ground.scalar_properties['reflectance'] = 0.72
    mat_ground.scalar_properties['transmission'] = 0.6
    mat_ground.scalar_properties['thickness'] = 0.3
    mat_ground.scalar_properties['absorption_distance'] = 0.1
    
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    
    bw_colors = 3*source_values/np.max(source_values)
    source_temp.colors = o3d.utility.Vector3dVector(np.asarray([bw_colors, bw_colors, bw_colors]).T)

    bw_colors = 3*target_values/np.max(target_values)
    target_temp.colors = o3d.utility.Vector3dVector(np.asarray([bw_colors, bw_colors, bw_colors]).T)
    
    target_temp = target_temp.uniform_down_sample(every_k_points=50)
    source_temp = source_temp.uniform_down_sample(every_k_points=50)
    print("pcd ok")
    source_temp = o3d.geometry.TetraMesh.create_from_point_cloud(source_temp)
    print("mesh created")
    #source_temp.material = mat_ground
    print("mesh material ok")

    target_temp = o3d.geometry.TetraMesh.create_from_point_cloud(target_temp)
    #target_temp.material = mat_ground
    

    # pick points from two point clouds and builds correspondences
    picked_id_source = pick_points(source_temp)
    picked_id_target = pick_points(target_temp)
    
    
    assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
    assert (len(picked_id_source) == len(picked_id_target))
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate transformation:
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint(with_scaling=True)
    trans_init = p2p.compute_transformation(source, target, o3d.utility.Vector2iVector(corr))
    source_transformed = copy.deepcopy(source)
    source_transformed.transform(trans_init)
    
    draw_registration_result(source_transformed, target, np.identity(4))
    
    return source_transformed


def pick_points(pcd):
    # These are used to suppress the printed output from Open3D whil epicking points:
    stdout_old = sys.stdout
    sys.stdout = StringIO()
    # Create Visualizer with editing:
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    # user picks points
    result = vis.run()  
    vis.destroy_window()
    # This restores the output:
    sys.stdout = stdout_old
    return vis.get_picked_points()

### test registration without color:

In [5]:
# Test registration:

# Read files and create point clouds
reference_fly_filename = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/References_and_masks/Reference_abdomen_binary_2_2_2.tif"
abdomen_mask_file = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/References_and_masks/Reference_abdomen_mask_thick_2_2_2.tif"

source_file_name = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/02_preprocessed/Preprocessed_C1-220320_D6xEn-Gal4_1.tif"
source_file_name_dsred = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/02_preprocessed/Preprocessed_C2-220320_D6xEn-Gal4_1.tif"

Reference_Image = io.imread(reference_fly_filename)
Abdomen_Mask = io.imread(abdomen_mask_file)
Source_Image = io.imread(source_file_name)
Source_dsred = io.imread(source_file_name_dsred)

#Source_TL = segmented_image_TL
source, source_values = image_to_pcd(Source_Image)
source_dsred, source_values_dsred = image_to_pcd(Source_dsred)

target, target_values = image_to_pcd(Reference_Image)

source = _manual_registration(source, source_values, target, target_values)

pcd ok
mesh created
mesh material ok


TypeError: add_geometry(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.visualization.Visualizer, geometry: open3d.cpu.pybind.geometry.Geometry, reset_bounding_box: bool = True) -> bool

Invoked with: VisualizerWithEditing with name Open3D, (TetraMesh with 28787 points and 186422 tetrahedra., [136, 884, 1206, 352, 1205, 600, 1194, 1552, 203, 1323, 434, 1692, 5784, 3523, 2105, 992, 2549, 697, 1, 0, 1961, 87, 632, 184, 830, 18991, 21657, 17611, 20340, 18992, 10075, 14761, 1694, 1948, 2843, 3324, 2844, 403, 1260, 6167, 4935, 8205, 2506, 1645, 680, 177, 6428, 9271, 7098, 437, 2414, 371, 3878, 1689, 10544, 827, 1520, 9272, 17612, 9266, 13296, 1693, 2100, 7097, 5174, 13349, 10994, 17629, 14787, 1899, 8124, 853, 335, 569, 329, 9354, 4594, 4602, 4605, 6441, 6435, 2552, 7466, 9688, 6785, 11395, 6788, 6132, 636, 189, 196, 693, 191, 4042, 27693, 7795, 9270, 204, 2671, 3132, 3623, 793, 882, 599, 879, 1963, 1965, 4941, 132, 372, 85, 28